# Create Machine learning model for public bike parking lots predictions

After Worked on data analytics, we will get started to build up machine learning models 

In [36]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows',5)
pd.set_option('large_repr', 'info')

li = []
for dirname, _, filenames in os.walk('./data/'):
    for filename in filenames:
        df = pd.read_csv(dirname+filename, index_col=None, header=0)
        li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

frame


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 814262 entries, 0 to 814261
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   _id              814262 non-null  object
 1   station_no       814262 non-null  int64 
 2   station_name     814262 non-null  object
 3   station_name_en  814262 non-null  object
 4   district         814262 non-null  object
 5   district_en      814262 non-null  object
 6   total            814262 non-null  int64 
 7   rentAvailable    814262 non-null  int64 
 8   returnAvaiable   814262 non-null  int64 
 9   updated_time     814262 non-null  object
 10  info_time        814262 non-null  object
dtypes: int64(4), object(7)
memory usage: 68.3+ MB

## Before we start building machine learning model
We have several things to do when build up model
- drop unused columns
- data type transformations
- leaving out the outliers
- split data into training set and test set

### Drop unused columns 
In the last page, we have identified the data with the type. 
we start to drop columns with serial numbers or their station name. 

In [37]:
frame.drop(columns=['_id', 'station_name','station_name_en','district','district_en'],inplace=True)

frame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 814262 entries, 0 to 814261
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   station_no      814262 non-null  int64 
 1   total           814262 non-null  int64 
 2   rentAvailable   814262 non-null  int64 
 3   returnAvaiable  814262 non-null  int64 
 4   updated_time    814262 non-null  object
 5   info_time       814262 non-null  object
dtypes: int64(4), object(2)
memory usage: 37.3+ MB

### Leaving out error values 
 

In [38]:
frame = frame.loc[(frame['total'] == frame['rentAvailable'] + frame['returnAvaiable']),:]

frame.drop(columns='station_no',inplace=True)

/var/folders/gw/xs4027fs03d38xr930y9wywm0000gn/T/ipykernel_57965/108571922.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame.drop(columns='station_no',inplace=True)


In [39]:

## convert datetime strings to datetime formats
frame['info_time'] = pd.to_datetime(frame['info_time'],format='%Y-%m-%d %H:%M:%S')
frame['updated_time'] = pd.to_datetime(frame['updated_time'],format='%Y-%m-%d %H:%M:%S')

## calculate time difference between station updated time and current time
timeDiff = (frame['info_time'] - frame['updated_time']) / pd.Timedelta(minutes=1)

frame['TimeDiff'] = timeDiff

/var/folders/gw/xs4027fs03d38xr930y9wywm0000gn/T/ipykernel_57965/721652273.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame['info_time'] = pd.to_datetime(frame['info_time'],format='%Y-%m-%d %H:%M:%S')
/var/folders/gw/xs4027fs03d38xr930y9wywm0000gn/T/ipykernel_57965/721652273.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame['updated_time'] = pd.to_datetime(frame['updated_time'],format='%Y-%m-%d %H:%M:%S')
/var/folders/gw/xs4027fs03d38xr930y9wywm0000gn/T/ipykernel_57965/721652273.py:8: Setti

In [40]:
## judge if current time is workday or not
def isWorkHour (timeSeries):
    work_day = timeSeries.weekday() < 5
    work_hour = timeSeries.hour >= 8 & timeSeries.hour < 19
    return work_day & work_hour

workday = frame['info_time'].apply(isWorkHour)

frame['isWorkDay'] = workday

frame

/var/folders/gw/xs4027fs03d38xr930y9wywm0000gn/T/ipykernel_57965/2541519960.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame['isWorkDay'] = workday


<class 'pandas.core.frame.DataFrame'>
Index: 707282 entries, 0 to 814261
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   total           707282 non-null  int64         
 1   rentAvailable   707282 non-null  int64         
 2   returnAvaiable  707282 non-null  int64         
 3   updated_time    707282 non-null  datetime64[ns]
 4   info_time       707282 non-null  datetime64[ns]
 5   TimeDiff        707282 non-null  float64       
 6   isWorkDay       707282 non-null  bool          
dtypes: bool(1), datetime64[ns](2), float64(1), int64(3)
memory usage: 38.4 MB

In [41]:
## Add ratio between rentAvailable and total
frame['ratio'] = frame['rentAvailable'] / frame['total']

frame

/var/folders/gw/xs4027fs03d38xr930y9wywm0000gn/T/ipykernel_57965/1343256383.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame['ratio'] = frame['rentAvailable'] / frame['total']


<class 'pandas.core.frame.DataFrame'>
Index: 707282 entries, 0 to 814261
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   total           707282 non-null  int64         
 1   rentAvailable   707282 non-null  int64         
 2   returnAvaiable  707282 non-null  int64         
 3   updated_time    707282 non-null  datetime64[ns]
 4   info_time       707282 non-null  datetime64[ns]
 5   TimeDiff        707282 non-null  float64       
 6   isWorkDay       707282 non-null  bool          
 7   ratio           707282 non-null  float64       
dtypes: bool(1), datetime64[ns](2), float64(2), int64(3)
memory usage: 43.8 MB